In [ ]:
# Marc Sorrentino 2024-08
# Model development for realtime spectral fitting and embedding

# Installations:

In [33]:
!pip install torch
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install torchinfo
!pip install torchsummary


# IMPORTS

In [1]:
import sys
sys.path.append('/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec')
# sys.path.append('/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec/Models')
from Models.UTime import USpec
from Assembly.SpecsDataset import SpecsDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
# from torchsummary import summary
import math
import glob
from count_parameters import count_parameters
from train import train 
import time

# ASSEMBLY PARAMETERS

In [2]:
import os
# Define assembly parameters
params = {}
params['startEpoch'] = 0
params['endEpoch'] = 50
params['local_rank'] = 0
params['modelName'] = "USpec"
params['numWorkers'] = 4
params['batchSize'] = 1
params['learningRate'] = 0.001
inventoryDir = "/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory"
dataDir = "/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Dataset"
checkpointDir = os.path.join(inventoryDir,params['modelName'])
os.makedirs(checkpointDir, exist_ok=True)
# count the number of 'training sessions' for a given model
numModelItrs = sum(1 for item in os.listdir(checkpointDir) if os.path.isdir(os.path.join(checkpointDir, item)))
params['checkpoint_dir']=os.path.join("/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory",params['modelName'],("T"+str(numModelItrs).rjust(2,'0')))    
params['log_dir']=params['checkpoint_dir']


# SETUP

In [3]:
# Fold settings (Cross Validation)
trainFolds = [1,2,3]
validFold = [4]
testFold = [5]
# Dataset instantiation
specsData_train = SpecsDataset(dataDir, trainFolds)
specsData_valid = SpecsDataset(dataDir, validFold)
specsData_test = SpecsDataset(dataDir, testFold)
# Dataloader instantiation
trainLoader = DataLoader(specsData_train, shuffle=False, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
validLoader = DataLoader(specsData_valid, shuffle=False, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
testloader = DataLoader(specsData_valid, shuffle=False, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
# Model instantiation
model = USpec()
modelSize = count_parameters(model)
print(modelSize)
# Test pass (size validation)
# # Create a sample input tensor with shape (batch_size, channels, length)
input_tensor = torch.randn(1, 1, 195)
# # Run the forward pass and print the shapes
start=time.time()
output = model(input_tensor)
stop=time.time()
lat=stop-start
print("Pass Latency: " + str(lat))
print(output)
# Model Summary
print(summary(model, input_size=(1, 1, 195)))
# Optimizer instantiation
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss(reduction='sum')

/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec/Assembly/SpecsDataset.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.index = index[index.iloc[:,1].str.contains(pattern)]
/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec/Assembly/SpecsDataset.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.index = index[index.iloc[:,1].str.contains(pattern)]
/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec/Assembly/SpecsDataset.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  self.index = index[index.iloc[:,1].str.contains(pattern)]
/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec/.conda/lib/python3.11/site-packages/torch/nn/modules/conv.py:304: UserWarning: Using padding='same' with even kernel lengths a

187833
torch.Size([1, 8, 96])
torch.Size([1, 16, 48])
torch.Size([1, 32, 24])
torch.Size([1, 64, 12])
torch.Size([1, 128, 12])
torch.Size([1, 64, 24])
torch.Size([1, 32, 48])
torch.Size([1, 16, 96])
torch.Size([1, 8, 192])
torch.Size([1, 1, 26])
Pass Latency: 0.009292125701904297
tensor([[[-0.2841, -0.3268, -0.2738, -0.1972, -0.1236, -0.0291, -0.0055,
          -0.1323, -0.2602, -0.1982, -0.0367, -0.1678, -0.3607, -0.4218,
          -0.3772, -0.5259, -0.6051, -0.4387, -0.3711, -0.3229, -0.2203,
          -0.3017, -0.3990, -0.3729, -0.4271, -0.2868]]],
       grad_fn=<ConvolutionBackward0>)
torch.Size([1, 8, 96])
torch.Size([1, 16, 48])
torch.Size([1, 32, 24])
torch.Size([1, 64, 12])
torch.Size([1, 128, 12])
torch.Size([1, 64, 24])
torch.Size([1, 32, 48])
torch.Size([1, 16, 96])
torch.Size([1, 8, 192])
torch.Size([1, 1, 26])
Layer (type:depth-idx)                   Output Shape              Param #
USpec                                    [1, 1, 26]                --
├─ModuleList: 1-16 

# TRAINING

In [4]:
# Model Training
model = train(params, model, trainLoader, validLoader, criterion, optimizer)

RuntimeError: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

# TESTING

In [ ]:
# Model Testing
model = test(params, model, testLoader, criterion, optimizer)